Escolha da task na pasta Training

In [0]:
TASK = 'assin-ptpt-sts'
DATA_DIR = 'data/canonical_data/'
MODEL_PATH = 'checkpoint/model_4.pt'
SCORE_PATH = 'score-{}.json'.format(TASK)
TEST_PATH = DATA_DIR  +  'bert_uncased_lower/'  + TASK + '_test.json'

Baixar o repositório do mt-dnn e requisitos

In [8]:
!git clone https://github.com/namisan/mt-dnn
%cd mt-dnn 
#!git checkout f444fe9109d
!pip3 install pytorch_pretrained_bert==0.5.1
!pip3 install sentencepiece 
!pip3 install seqeval  
!pip3 install tensorboardX

Cloning into 'mt-dnn'...
remote: Enumerating objects: 1577, done.
remote: Total 1577 (delta 0), reused 0 (delta 0), pack-reused 1577
Receiving objects: 100% (1577/1577), 994.86 KiB | 13.82 MiB/s, done.
Resolving deltas: 100% (891/891), done.
/content/mt-dnn
     |████████████████████████████████| 92kB 2.9MB/s 
     |████████████████████████████████| 1.0MB 2.8MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=49fdd3856db69ec23d26b2a00f543ca8c033413945c00947c9704b35f2da1dca
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
     |████████████████████████████████| 204kB 2.8MB/s 


Baixar modelos de rede neural

In [9]:
!bash download.sh

Create a folder BERT_DIR
--2020-03-10 19:43:50--  https://mrc.blob.core.windows.net/mt-dnn-model/bert_model_base_v2.pt
Resolving mrc.blob.core.windows.net (mrc.blob.core.windows.net)... 52.190.240.132
Connecting to mrc.blob.core.windows.net (mrc.blob.core.windows.net)|52.190.240.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 437961819 (418M) [application/octet-stream]
Saving to: ‘/content/mt-dnn/mt_dnn_models/bert_model_base_uncased.pt’

/content/mt-dnn/mt_ 100%[===================>] 417.67M  17.3MB/s    in 23s     

2020-03-10 19:44:14 (18.0 MB/s) - ‘/content/mt-dnn/mt_dnn_models/bert_model_base_uncased.pt’ saved [437961819/437961819]

--2020-03-10 19:44:14--  https://mrc.blob.core.windows.net/mt-dnn-model/bert_model_large_v2.pt
Resolving mrc.blob.core.windows.net (mrc.blob.core.windows.net)... 52.190.240.132
Connecting to mrc.blob.core.windows.net (mrc.blob.core.windows.net)|52.190.240.132|:443... connected.
HTTP request sent, awaiting response... 200 O

Baixar dados da task

In [10]:
!mkdir data/canonical_data

mkdir: cannot create directory ‘data’: File exists
Cloning into 'mtdnn_port'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 454 (delta 110), reused 114 (delta 57), pack-reused 277
Receiving objects: 100% (454/454), 63.12 MiB | 16.62 MiB/s, done.
Resolving deltas: 100% (255/255), done.


Aplicar patch em metrics.py para incluir *f1_macro* e *mse*

In [11]:
!patch data_utils/metrics.py < ../Training/metrics.patch

patching file data_utils/metrics.py


Mover dados de entrada

In [0]:
!bash move_input.sh $TASK

Preprocess data

In [53]:
!python prepro_std.py --do_lower_case --root_dir data/canonical_data --task_def {TASK}.yaml


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
03/10/2020 09:13:06 Task assin2-sts
03/10/2020 09:13:07 data/canonical_data/bert_uncased_lower/assin2-sts_train.json
03/10/2020 09:13:09 data/canonical_data/bert_uncased_lower/assin2-sts_dev.json
03/10/2020 09:13:09 data/canonical_data/bert_uncased_lower/assin2-sts_test.json


Corrigir código do *predict.py*

In [0]:
!patch predict.py < ../Training/predict.patch

Predict task

In [99]:
!python predict.py --task {TASK} --task_def {TASK}.yaml --with_label --checkpoint {MODEL_PATH} --prep_input {TEST_PATH} --score {SCORE_PATH}

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Loaded 2448 samples out of 2448
predicting 0
predicting 100
predicting 200
predicting 300
{'Pearson': 85.16350856025014}


Abrir *Tensorboard*.

Executa no *Chrome* normalmente, mas não funciona no *Firefox*

In [0]:
%load_ext tensorboard
%tensorboard  --logdir tensorboard_logdir